In [145]:
from pathlib import Path
from adept.config import DATA_DIR, ROOT_DIR, logger, ASSETS_DIR

import pandas as pd
import re
import numpy as np
import numbers

In [22]:
data_dir = Path('.')

file_name = "BRYOTraits for Silvia.xlsx"

df = pd.read_excel(data_dir / file_name, sheet_name = 'SP_traits_AZO', header=None)

In [7]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,1058,1059,1060,1061,1062,1063,1064,1065,1066,1067
0,NaN,TAXONOMY & NOMENCLATURE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# The first rows are taxonomic data, so lets drop it
df = df.iloc[: , 3:]

first_col = df[:1].apply(pd.Series.first_valid_index, axis=1).loc[0]

17


In [26]:
df2 = df.iloc[: , first_col - 3:]

In [31]:
df2.head()

,17,18,19,20,21,22,23,24,25,26,...,1058,1059,1060,1061,1062,1063,1064,1065,1066,1067
0,GEOGRAPHIC ATTRIBUTES & NATIVE STATUS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,European distribution,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [274]:
# for column in df2.columns:
#     print(nt(i))

def is_measurement(cols):
    return any([bool(re.search(r'\(mm|cm|μm|µm\)', col)) for col in cols if not pd.isnull(col)])

ancestors = {}
data = {}
header_rows = 8

for i in range(17, len(df.columns)):
    headers = df2[i][:header_rows].values
    
    # Once we get to Bibliography, exit     
    if 'Bibliography' in headers:
        break
    
    # Header row that potentially holds the trait value
    # But may also be a header, or null     
    trait = df2[i].loc[header_rows]
    
    for j, header in enumerate(headers):
                
        if not pd.isnull(header):
                                     
            # If header has a value, reset all subsequent ancestors 
            ancestors = {k: v for k, v in ancestors.items() if k < j}
            
            ancestors[j] = header
            
            # ancestors = ancestors[:j]
            # ancestors.append(header)
           
        
    field_name_parts = list(ancestors.values())

    # col_values = df2.loc[header_rows+1:, i].values
    is_measurement_col = is_measurement([field_name_parts[-1], trait])
        
    if is_measurement_col:
        
        # If trait has a value, replace the last column name
        if not pd.isnull(trait):
            field_name_parts = field_name_parts[:-1]
            field_name_parts.append(trait)
            
        trait = 'MEASUREMENT'            
        
    
    elif pd.isnull(trait):                
        # trait = 'TEXT' 
        pass

    field_name = '.'.join(field_name_parts)

    data.setdefault(field_name, set()).add(trait)        

# 
new_df = pd.DataFrame(list(data.values()), index=data.keys())
    
new_df.T.head(n=40)
    # print(headers)

# df2.loc[17]

# df2[17][0:9]

# for i in range(0, 9):
#     print(i)

,GEOGRAPHIC ATTRIBUTES & NATIVE STATUS.European distribution.Major biome,GEOGRAPHIC ATTRIBUTES & NATIVE STATUS.European distribution.Eastern limit,"GEOGRAPHIC ATTRIBUTES & NATIVE STATUS.Occurrence in the Azores \n(Gabriel et al., 2010).Western group","GEOGRAPHIC ATTRIBUTES & NATIVE STATUS.Occurrence in the Azores \n(Gabriel et al., 2010).Central group","GEOGRAPHIC ATTRIBUTES & NATIVE STATUS.Occurrence in the Azores \n(Gabriel et al., 2010).Eastern group","GEOGRAPHIC ATTRIBUTES & NATIVE STATUS.Native status\n(Gabriel et al., 2010)","RARITY.IUCN conservation status\n(ECCB, 1995)",ADAPTIVE STRATEGIES.Life Strategy.r strategists,ADAPTIVE STRATEGIES.Life Strategy.K strategists,ADAPTIVE STRATEGIES.Life forms,...,MORPHOLOGY.Sporophyte.CAPSULE.WHOLE CAPSULE.Width max (mm),MORPHOLOGY.Sporophyte.CAPSULE.WHOLE CAPSULE.Shape,MORPHOLOGY.Sporophyte.CAPSULE.WHOLE CAPSULE.Symmetry,MORPHOLOGY.Sporophyte.CAPSULE.WHOLE CAPSULE.Inclination,MORPHOLOGY.Sporophyte.CAPSULE.WHOLE CAPSULE.Colour,MORPHOLOGY.Sporophyte.CAPSULE.WHOLE CAPSULE.Striation,MORPHOLOGY.Sporophyte.CAPSULE.CAPSULE PARTS.Operculum.Presence,MORPHOLOGY.Sporophyte.CAPSULE.CAPSULE PARTS.Operculum.Concavity,MORPHOLOGY.Sporophyte.CAPSULE.CAPSULE PARTS.Operculum.Shape,MORPHOLOGY.Sporophyte.CAPSULE.CAPSULE PARTS.Peristome.Presence
0,Southern-temperate,Circumpolar,Corvo,Terceira,Santa Maria,NaN,NaN,colonist,dominant,annuals,...,MEASUREMENT,cylindrical,asymmetrical,pendulous,purplish-red,furrowed,presence,umbonate,beaked,presence
1,Artic-montane,Oceanic,Flores,Faial,São Miguel,None,None,short-lived shuttle,competitive perennial,pendants,...,None,elliptical,symmetrical,inclined,brown,sulcate,absence,convex,rostrate,absence
2,Temperate,Eurasian,None,Pico,None,None,None,annual shuttle,stress-tolerant perennial,turfs,...,None,globose,None,cernuous,reddish-brown,striate,None,concave,subulate,None
3,Boreal-montane,Suboceanic,None,São Jorge,None,None,None,fugitive,perennial stayer,mats,...,None,pyriform,None,erect,green,smooth,None,None,obtuse,None
4,Boreo-artic montane,Eurosiberian,None,Graciosa,None,None,None,pioneer colonist,long-lived shuttle,cushions,...,None,strumose,None,oblique,yellow / yellowish,None,None,None,conical,None
5,Wide-temperate,Hyperoceanic,None,None,None,None,None,ephemeral colonist,None,tails,...,None,subcylindrical,None,horizontal,red / reddish,None,None,None,acute,None
6,Mediterranean-Atlantic,European,None,None,None,None,None,None,None,wefts,...,None,elongate,None,None,orange,None,None,None,rostellate,None
7,Wide-boreal,None,None,None,None,None,None,None,None,dendroids,...,None,suberect,None,None,yellowish-brown,None,None,None,apiculate,None
8,Boreo-temperate,None,None,None,None,None,None,None,None,fans,...,None,urceolate,None,None,None,None,None,None,None,None
9,None,None,None,None,None,None,None,None,None,None,...,None,obpyriform,None,None,None,None,None,None,None,None


In [275]:
new_df.T.to_excel(data_dir / 'bryo.xlsx')

In [75]:
x = x[2]

In [76]:
x

[1, 2]

In [99]:
df.head()

,3,4,5,6,7,8,9,10,11,12,...,1058,1059,1060,1061,1062,1063,1064,1065,1066,1067
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [206]:
99

col_values = df2.loc[header_rows+1:, 99].values
is_numeric_col = all([is_numeric(v) for v in col_values if not pd.isnull(v)])

print(is_numeric_col)
 

True


True

In [166]:
# for a,s in zip(values, is_numeric):
#     print(a)
#     print(s)
#     pd.isnull(s)

# is_numeric = all([str(v).isnumeric() for v in values if not pd.isnull(v)])
# print(is_numeric)

# # import numbers

# # v2 = [v for v in values if not pd.isnull(v)]
# # v2
for v in values:
    if not str(v).isdigit():
        print(v)
    

7.5
10.5
15.5
14.5
12.5


In [244]:
for k,v in data.items():
    if re.search(r'\(mm|cm|μm|µm\)', k):
        # print(k)
        pass
    else:
        pass
        if '(' in k:
            print(k)
        # print(k)

# x = 'MORPHOLOGY.Gametophyte.Gametophore.Maximum width (mm)'


# # re.match(r'mm', ):

GEOGRAPHIC ATTRIBUTES & NATIVE STATUS.Occurrence in the Azores 
(Gabriel et al., 2010).Western group
GEOGRAPHIC ATTRIBUTES & NATIVE STATUS.Occurrence in the Azores 
(Gabriel et al., 2010).Central group
GEOGRAPHIC ATTRIBUTES & NATIVE STATUS.Occurrence in the Azores 
(Gabriel et al., 2010).Eastern group
GEOGRAPHIC ATTRIBUTES & NATIVE STATUS.Native status
(Gabriel et al., 2010)
RARITY.IUCN conservation status
(ECCB, 1995)
MORPHOLOGY.Gametophyte.Spores.Colour
(TINT: P = pale; B = bright; D = dark)
MORPHOLOGY.Gametophyte.Gametophore.Colour
(TINT: P = pale; B = bright; D = dark)
MORPHOLOGY.Gametophyte.Gametophore.Vegetative propagules.Colour
(TINT: P = pale; B = bright; D = dark)
MORPHOLOGY.Gametophyte.Gametophore.STEM.WHOLE STEM.Colour
(TINT: P = pale; B = bright; D = dark)
MORPHOLOGY.Gametophyte.Gametophore.RHIZOIDS.Colour
(TINT: P = pale; B = bright; D = dark)
MORPHOLOGY.Sporophyte.CAPSULE.WHOLE CAPSULE.Position (in relation to perichaetium)
